# GOM create data

## Author: M.Ravasi

### Author: M.Ravasi

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from mcslopes.preprocessing import butter_lowpass_filter, gradient_data

## Parameters

In [2]:
# data
dt = 4e-3 # time sampling (in s)
dx = 23 # spatial sampling (in m)
x0 = 100 # initial offset

# pre-processing
fcut = 35. # cut-off frequency of low pass filter  (in Hz)
ncut = 11 # lenght of low pass filter (in Hz)
tdir = 430 # index above which the data will be removed - this is a rough way to remove the direct arrival
nfft_t, nfft_x = 2**11, 2**9 # lenght of frequency and wavenumber axis (used to compute gradients and perform FK transforms during data reconstruction)

# subsampling
nsub = 5

In [3]:
# Load data
forig = np.load('../../data/gom_data.npz')
dorig = forig['data']
x = forig['x']
t = forig['t']
nx = x.size

# Filter to remove spatial aliasing
dorig = butter_lowpass_filter(dorig.T, fcut, 1/dt, ncut).T

# Remove direct arrival
dorig[:tdir] = 0.

# Ensure to have first and last trace
x = x[:(nx//nsub)*nsub-nsub+1]
dorig = dorig[:, :(nx//nsub)*nsub-nsub+1]

# Spatial derivatives
d1, d2 = gradient_data(dorig.T, nfft_x, nfft_t, dx, dt)[:2]
d1, d2 = d1.T, d2.T

np.savez(f'../../data/gom_data_forsub{nsub}.npz', data=dorig, grad1=d1, grad2=d2, x=x, t=t)

In [4]:
# Subsample
dsub = dorig[:, ::nsub] 
d1sub = d1[:, ::nsub] 
d2sub = d2[:, ::nsub] 

np.savez(f'../../data/gom_data_sub{nsub}.npz', data=dsub, grad1=d1sub, grad2=d2sub, xorig=x, x=x[::nsub], t=t)